In [21]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [2]:
max_sen_len = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1
EMBEDDING_FILE = '/media/ashish/New Volume/M.Tech/Thesis/Dataset/Thesis2/scripts/reader/data/embeddings/glove.840B.300d.txt'

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [4]:
train_df = pd.read_csv('input/imdb/labeledTrainData.tsv', sep='\t')

In [5]:
texts, labels = [], []
for i in range(train_df['review'].shape[0]):
    text = train_df['review'][i]
    texts.append(clean_str(text))
    labels.append(train_df['sentiment'][i])

In [6]:
labels = np.array(labels)

In [7]:
tok = Tokenizer(num_words=MAX_NUM_WORDS)
tok.fit_on_texts(texts)
seq = tok.texts_to_sequences(texts)
data = pad_sequences(seq,maxlen=max_sen_len)

In [8]:
data = np.expand_dims(data, axis=1)
# data.shape
# max_doc = data.shape[0]

In [9]:
idx = tok.word_index
labels_cat = np.expand_dims(labels,axis=1)
# labels_cat = to_categorical(np.asarray(labels))

In [19]:
x_train,x_val,y_train,y_val = train_test_split(data,labels_cat,test_size=0.12)

In [12]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [13]:
num_words = min(MAX_NUM_WORDS, len(idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in idx.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [16]:
from model import *
model = createHierarchicalAttentionModel(max_sen_len, embWeights=embedding_matrix,
                                                        embeddingSize = 300, vocabSize = min(MAX_NUM_WORDS, len(idx) + 1))

/media/ashish/New Volume/Documents/Compressed/ML/Codes/Courses/Hierarchical_Text_Classification/model.py:156: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [23]:
filepath="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
callbacks_list = [checkpoint, early]

In [24]:
model.fit(x_train, y_train, batch_size=64, epochs=4,validation_data=(x_val, y_val),callbacks = callbacks_list,verbose=1)

Train on 22000 samples, validate on 3000 samples
Epoch 1/4
22000/22000 [==============================] - 103s 5ms/step - loss: 0.1440 - acc: 0.9517 - val_loss: 0.1516 - val_acc: 0.9477

Epoch 00001: val_acc improved from -inf to 0.94767, saving model to weights_base.best.hdf5
Epoch 2/4
22000/22000 [==============================] - 105s 5ms/step - loss: 0.1069 - acc: 0.9632 - val_loss: 0.1667 - val_acc: 0.9487

Epoch 00002: val_acc improved from 0.94767 to 0.94867, saving model to weights_base.best.hdf5
Epoch 3/4
22000/22000 [==============================] - 104s 5ms/step - loss: 0.0781 - acc: 0.9747 - val_loss: 0.1989 - val_acc: 0.9353

Epoch 00003: val_acc did not improve
Epoch 4/4
22000/22000 [==============================] - 103s 5ms/step - loss: 0.0627 - acc: 0.9796 - val_loss: 0.2265 - val_acc: 0.9227

Epoch 00004: val_acc did not improve


In [25]:
# print('score ',score)
# print('score ',loss)
model.load_weights(filepath)

In [26]:
test_df = pd.read_csv('input/imdb/testData.tsv', sep='\t')

In [27]:
texts_test = [] 
for i in range(test_df['review'].shape[0]):
    text = test_df['review'][i]
    texts_test.append(clean_str(text))

In [28]:
test_seq = tok.texts_to_sequences(texts_test)
test_data = pad_sequences(test_seq,maxlen=max_sen_len)
test_data = np.expand_dims(test_data, axis=1)

In [29]:
pred = model.predict(test_data,batch_size=1024)
print('pred ',pred)

pred  [[9.9904817e-01]
 [3.6834122e-03]
 [4.4308049e-03]
 ...
 [4.3118285e-04]
 [9.9770278e-01]
 [4.5535466e-01]]


In [30]:
final_pred = []
for i in pred:    
    if i>0.9:
        final_pred.append(1)
    else: final_pred.append(0)        

In [31]:
subm = pd.DataFrame(data=[],columns=['id','sentiment'])#read_csv('sampleSubmission.csv')
subm['sentiment'] = final_pred
subm['id'] = test_df['id']
subm.to_csv('submission.csv',index=False)